In [11]:
import pandas as pd
import os


In [12]:
prior_path = os.path.join('Resources', 'order_products__prior.csv')
train_path = os.path.join('Resources', 'order_products__train.csv')

order_path = os.path.join('Resources', 'orders2.csv')

In [13]:
prior_df = pd.read_csv(prior_path)
train_df = pd.read_csv(train_path)
order_df = pd.read_csv(order_path) 

1048575


104042

In [43]:
order_products_df = pd.concat([prior_df, train_df])
order_products_df

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0
...,...,...,...,...
1048570,2593147,44142,3,0
1048571,2593147,34969,4,1
1048572,2593147,33055,5,0
1048573,2593147,16185,6,1


In [78]:
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from app_config import USER, PASSWORD, HOST, PORT, DATABASE, DIALECT, DRIVER, secret
from sqlalchemy import Column, Integer, String, BigInteger, VARCHAR, SmallInteger
from sqlalchemy import *

SQALCHEMY_DATABASE_URL = f"{DIALECT}+{DRIVER}://{USER}:{PASSWORD}@{HOST}/local_inventory"

engine = create_engine(
    SQALCHEMY_DATABASE_URL, 
    pool_recycle=3600,
    pool_pre_ping=True
)

Base = declarative_base()
# metadata  = MetaData()

# engine.execute('USE inventory_test')

# products = Table("products", metadata,
#     Column('product_id', BigInteger, primary_key=True),
#     Column('product_name', VARCHAR (120)),
#     Column('aisle_id', Integer, ForeignKey('aisles.aisles_id')),
#     Column('department_id', Integer, ForeignKey('departments.department_id')),
#     Column('price', Integer)
#                )


# orders = Table("orders", metadata,
#     Column('order_id', BigInteger, primary_key=True),
#     Column('user_id', BigInteger, ForeignKey('users.user_id')),
#     Column('order_dow', BigInteger),
#     Column('days_since_prior_order', Integer)
#              )

# order_products = Table("order_products", metadata,
#     Column('order_id', BigInteger, primary_key=True),
#     Column('product_id', BigInteger, ForeignKey('products.product_id')),
#     Column('add_to_cart_order', Integer),
#     Column('reordered', Integer),
#                      )

# departments = Table("departments", metadata,
#     Column('department_id', BigInteger, primary_key=True),
#     Column('department', VARCHAR(30))
#                   )


# aisle = Table("aisles", metadata,
#     Column('aisle_id', BigInteger, primary_key=True),
#     Column('aisle', VARCHAR(30))
#              )
class DictMixIn:
    def to_dict(self):
        return {
            column.name: getattr(self, column.name)
            if not isinstance(getattr(self, column.name), datetime.datetime)
            else getattr(self, column.name).isoformat()
            for column in self.__table__.columns
        }
class User(Base):
    extend_existing=True
    __tablename__ = "users" 
    id = Column(Integer, primary_key=True, autoincrement=True)
    username = Column(String(15), unique=True)
    email = Column(String(50), unique=True)
    passw = Column(String(80))

class Products(Base, DictMixIn):
    __tablename__ = "products"
    product_id = Column(BigInteger, primary_key=True)
    product_name = Column(VARCHAR (120))
    aisle_id = Column(Integer, ForeignKey('aisles.aisle_id'))
    department_id = Column(Integer, ForeignKey('departments.department_id'))
    price = Column(Integer)


class Orders(Base, DictMixIn):
    __tablename__ = "orders"
    order_id = Column(BigInteger, primary_key=True)
    user_id = Column(BigInteger, ForeignKey('users.id'))
    order_dow = Column(BigInteger)
    days_since_prior_order = Column(Integer)

class Order_products(Base, DictMixIn):
    __tablename__ = "order_products"
    order_id = Column(BigInteger, primary_key=True )
    product_id = Column(BigInteger, ForeignKey('products.product_id'))
    num_of_product = Column(Integer)
    add_to_cart_order = Column(Integer)
    reordered = Column(SmallInteger)

class Departments(Base, DictMixIn):
    __tablename__ = "departments"
    department_id = Column(BigInteger, primary_key=True)
    department = Column(VARCHAR(30))


class Aisles(Base, DictMixIn):
    __tablename__ = "aisles"
    aisle_id = Column(BigInteger, primary_key=True)
    aisle = Column(VARCHAR(30))

Base.metadata.create_all(engine)

NoReferencedTableError: Foreign key associated with column 'orders.user_id' could not find table 'users' with which to generate a foreign key to target column 'user_id'

In [76]:

# engine.execute('create table Products_prior')
# engine.execute('create table Products_train')
# engine.execute('create table Orders')
order_products_df.to_sql(name='order_products', con=engine, if_exists='replace', index=False)
order_df.to_sql(name='Orders', con=engine, if_exists='replace', index=False)

MySQLInterfaceError: MySQL server has gone away